In [51]:
#import liabraries
import string
import re
import codecs
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
from sklearn import feature_extraction
from sklearn import linear_model
from sklearn import pipeline
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [59]:
eng_df = pd.read_csv("eng_news_2015_10K-sentences.txt","utf-8", header=None, names=["English"])
ger_df = pd.read_csv("deu_mixed-typical_2011_10K-sentences.txt","utf-8", header=None, names=["German"])
fre_df = pd.read_csv("fra_mixed_2009_10K-sentences.txt","utf-8", header=None, names=["French"])
spa_df = pd.read_csv("spa_news_2011_10K-sentences.txt","utf-8", header=None, names=["Spanish"])
chi_df = pd.read_csv("zho_news_2007-2009_10K-sentences.txt", header=None, names=["Chinese"])

<ipython-input-59-d8d41f0c416c>:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  eng_df = pd.read_csv("eng_news_2015_10K-sentences.txt","utf-8", header=None, names=["English"])
<ipython-input-59-d8d41f0c416c>:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  ger_df = pd.read_csv("deu_mixed-typical_2011_10K-sentences.txt","utf-8", header=None, names=["German"])
<ipython-input-59-d8d41f0c416c>:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by sp

In [16]:
#get punctuation
for char in string.punctuation:
    print(char, end=' ')
translate_table = dict((ord(char), None) for char in string.punctuation)

! " # $ % & ' ( ) * + , - . / : ; < = > ? @ [ \ ] ^ _ ` { | } ~ 

In [17]:
data_eng = []
lang_eng = []
for i,line in eng_df.iterrows():
    line = line['English']
    if len(line) != 0:
        line = line.lower()
        line = re.sub(r"\d+","",line)
        line = line.translate(translate_table)
        data_eng.append(line)
        lang_eng.append("English")

In [18]:
data_ger = []
lang_ger = []
for i,line in ger_df.iterrows():
    line = line['German']
    if len(line) != 0:
        line = line.lower()
        line = re.sub(r"\d+","",line)
        line = line.translate(translate_table)
        data_ger.append(line)
        lang_ger.append("German")

In [19]:
data_fre = []
lang_fre = []
for i,line in fre_df.iterrows():
    line = line['French']
    if len(line) != 0:
        line = line.lower()
        line = re.sub(r"\d+","",line)
        line = line.translate(translate_table)
        data_fre.append(line)
        lang_fre.append("French")

In [20]:
data_spa = []
lang_spa = []
for i,line in spa_df.iterrows():
    line = line['Spanish']
    if len(line) != 0:
        line = line.lower()
        line = re.sub(r"\d+","",line)
        line = line.translate(translate_table)
        data_spa.append(line)
        lang_spa.append("Spanish")

In [21]:
data_chi = []
lang_chi = []
for i,line in chi_df.iterrows():
    line = line['Chinese']
    if len(line) != 0:
        line = line.lower()
        line = re.sub(r"\d+","",line)
        line = re.sub(r"[a-zA-Z]+","", line)
        line = line.translate(translate_table)
        data_chi.append(line)
        lang_chi.append("Chinese")

In [27]:
df = pd.DataFrame({"Text":data_eng+data_ger+data_fre+data_spa+data_chi,"Language":lang_eng+lang_ger+lang_fre+lang_spa+lang_chi})
df.shape

(49958, 2)

In [28]:
X, y = df.iloc[:,0],df.iloc[:,1]
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=0)

In [31]:
# vectorizer and model fitting pipeline
vectorizer = feature_extraction.text.TfidfVectorizer(ngram_range=(1,3), analyzer='char')
pipe_lr_r13 = pipeline.Pipeline([
    ('vectorizer', vectorizer),
    ('clf', linear_model.LogisticRegression())
])
# model fitting
pipe_lr_r13.fit(X_train, y_train)

Pipeline(steps=[('vectorizer',
                 TfidfVectorizer(analyzer='char', ngram_range=(1, 3))),
                ('clf', LogisticRegression())])

In [39]:
#model prediction
y_predict = pipe_lr_r13.predict(X_test)
#model evaluation
acc = (metrics.accuracy_score(y_test, y_predict))*100
print("Accuracy: ",acc,"%")


Accuracy:  99.81985588470776 %


In [56]:
matrix = metrics.confusion_matrix(y_test, y_predict)
print("Confusion Matrix: \n", matrix)


Confusion Matrix: 
 [[2022    0    0    0    0]
 [   0 1984    1    3    1]
 [   0    2 1990    5    2]
 [   0    2    1 2013    0]
 [   0    0    0    1 1965]]


In [57]:
#save the model
lrFile = open("LRModel.pck1", "wb")
pickle.dump(pipe_lr_r13, lrFile)
lrFile.close()